A cell for reading the data in spider-man.csv

# 1-2

In [1]:
using JuMP
using HiGHS

In [66]:
model = Model(HiGHS.Optimizer)
@variable(model, p1 >= 0)
@variable(model, p2 >= 0)
@objective(model, Min, 4*p1 + p2)
@constraint(model, 3*p1 + p2 >= 10)
@constraint(model, p1 + p2 >= 5)
@constraint(model, p1 >= 3)
set_silent(model)
optimize!(model)

In [3]:
println("Number of hours for process 1: ", value(p1))
println("Number of hours for process 2: ", value(p2))

Number of hours for process 1: 3.0
Number of hours for process 2: 2.0


# 1-5

In [ ]:
# import Pkg; Pkg.add("DataFrames"); Pkg.add("CSV"); Pkg.add("NamedArrays")
using DataFrames, CSV, NamedArrays
df = CSV.read("spider-man.csv", DataFrame, delim =',') 

# Processes are column names starting at the 3rd
J = propertynames(df)[3:end]

# Chemicals are the first column starting at the second row
I = convert(Array, df[2:end,1])

# Cost
c = Dict(zip(J,df[1,3:end]))

# Amt. required
b = Dict(zip(I,df[2:end,2]))

# Matrix of data 
A = Matrix(df[2:end,3:end])

# Make it a Named Array
A_NA = NamedArray(A, (I,J), ("Chemical", "Process")) 

In [70]:
# As anyone who watches the Simpsons knows, Spider-Pig is infinitely more powerful than Spider-man. Spider-Pig has a large chemical process plant to produce its special snout fluid. To produce snout fluid, he requires a set of chemicals I, and he can produce these chemicals by running of a collection of processes J. Running process j ∈ J for one hour costs cj, and produces aij vials of chemical i ∈ I. Snout fluid requires bi vials of chemical i ∈ I.
# Write a general linear optimization model that will inform Spider-Pig the number of hours of each process to run to minimize its cost while making enough vials of each required chemical.
model = Model(HiGHS.Optimizer)
@variable(model, p[J] >= 0)
@objective(model, Min, sum(c[j]*p[j] for j in J))
@constraint(model, [i in I], sum(A_NA[i,j]*p[j] for j in J) >= b[i])
set_silent(model)
optimize!(model)
println("Termination status: ", termination_status(model))
println("Objective value: ", objective_value(model))

Termination status: OPTIMAL
Objective value: 1433.717374299214


# 2-1

In [71]:
# # min 2x1 − x2 + 3x3 − 2x4
# # s.t. 3x2 − x3 + 4x4 ≥ 10 
# # 4x1 − 7x2 + x3 − x4 = 5
# # x1 − 3x3 + 8x4 ≤ 3 
# # x1, x4 ≥ 0
# # x2, x3 Unrestricted in Sign (Free)

model = Model(HiGHS.Optimizer)
@variable(model, x[1:4])
@objective(model, Min, 2*x[1] - x[2] + 3*x[3] - 2*x[4])
@constraint(model, 3*x[2] - x[3] + 4*x[4] >= 10)
@constraint(model, 4*x[1] - 7*x[2] + x[3] - x[4] == 5)
@constraint(model, x[1] - 3*x[3] + 8*x[4] <= 3)

@constraint(model, x[1] >= 0)
@constraint(model, x[4] >= 0)
set_silent(model)
optimize!(model)

println("Optimal solution: ", value.(x))
println("Optimal solution value: ", objective_value(model))
println("Termination status: ", termination_status(model))


Optimal solution: [7.593750000000001, 3.84375, 1.5312500000000007, -0.0]
Optimal solution value: 15.937500000000004
Termination status: OPTIMAL


# 2-3

In [72]:
# Solve in standard form
x = [:x1, :a, :b, :c, :d, :x4]
c = [-2, 1, -1, -3, 3, 2]
A = [  0  -3   3   1  -1  -4;
       4  -7   7   1  -1  -1;
      -4   7  -7  -1   1   1;
       1   0   0  -3   3   8 ]
b = [-10, 5, -5, 3]
model = Model(HiGHS.Optimizer)
@variable(model, x[1:6] >= 0)
@objective(model, Max, c'*x)
@constraint(model, A*x <= b)
set_silent(model)
optimize!(model)
println("Optimal value of decision variables: ", value.(x))
println("Optimal solution value: ", objective_value(model)) 


Optimal value of decision variables: [7.593750000000002, 3.84375, 0.0, 1.5312500000000007, 0.0, 0.0]
Optimal solution value: -15.937500000000005


# 4-2

In [73]:
using JuMP, HiGHS
model = Model(HiGHS.Optimizer)
@variable(model, x[1:4] >= 0)  # Raw sales
@variable(model, y[1:4] >= 0)  # Blended into Avgas A
@variable(model, z[1:4] >= 0)  # Blended into Avgas B

PN = [107, 93, 87, 108]
RVP = [5, 8, 4, 21]
Production = [3814, 2666, 4016, 1300]

# Objective function: Maximize profit
@objective(model, Max, 
    4.83 * sum(x) + 6.45 * sum(y) + 5.91 * sum(z)
)

for i in 1:4
    @constraint(model, x[i] + y[i] + z[i] <= Production[i])
end
# Avgas A Performance Number constraint
@constraint(model, sum(PN[i] * y[i] for i in 1:4) >= 100 * sum(y))
# Avgas B Performance Number constraint
@constraint(model, sum(PN[i] * z[i] for i in 1:4) >= 91 * sum(z))
@constraint(model, sum(RVP[i] * y[i] for i in 1:4) <= 7 * sum(y))
@constraint(model, sum(RVP[i] * z[i] for i in 1:4) <= 7 * sum(z))
set_silent(model)
optimize!(model)
println("Optimal profit: ", objective_value(model))
println("Optimal amounts sold raw: ", value.(x))
println("Optimal amounts blended into Avgas A: ", value.(y))
println("Optimal amounts blended into Avgas B: ", value.(z))


Optimal profit: 73879.38
Optimal amounts sold raw: [0.0, 0.0, 0.0, 85.0]
Optimal amounts blended into Avgas A: [3754.0, 2666.0, 920.0, 543.0]
Optimal amounts blended into Avgas B: [59.99999999999997, 0.0, 3096.0, 672.0]


# 5-2

In [62]:
using JuMP
using HiGHS

# Data (example values - replace with actual data)
E = ["C", "Cu", "Mn"]
J = ["F", "U", "E", "L", "O", "N", "M"]

# Parameters
cost = Dict("F" => 3, "U" => 2, "E" => 4, "L" => 1, "O" => 5, "N" => 2, "M" => 3)
avail = Dict("F" => 100, "U" => 200, "E" => 150, "L" => 300, "O" => 50, "N" => 100, "M" => 80)
gamma = Dict(
    "C" => Dict("F" => 2.5, "U" => 3.0, "E" => 1.5, "L" => 0.0, "O" => 0.4, "N" => 0.0, "M" => 0.0),
    "Cu" => Dict("F" => 0.0, "U" => 0.3, "E" => 0.5, "L" => 0.9, "O" => 0.9, "N" => 0.4, "M" => 0.6),
    "Mn" => Dict("F" => 1.3, "U" => 1.8, "E" => 1.5, "L" => 0.0, "O" => 1.1, "N" => 1.2, "M" => 0.0)
)
alpha = Dict("C" => 2.0, "Cu" => 0.4, "Mn" => 1.2)
beta = Dict("C" => 3.0, "Cu" => 0.6, "Mn" => 1.65)

# Model
model = Model(HiGHS.Optimizer)
# Variables
@variable(model, x[j in J] >= 0)
# Objective: Minimize cost
@objective(model, Min, sum(cost[j] * x[j] for j in J))
# Constraints
@constraint(model, sum(x[j] for j in J) == 500)  # Total steel
@constraint(model, [e in E], sum(gamma[e][j] * x[j] for j in J) >= alpha[e] * 500)  # Min element
@constraint(model, [e in E], sum(gamma[e][j] * x[j] for j in J) <= beta[e] * 500)   # Max element
@constraint(model, [j in J], x[j] <= avail[j])  # Availability

set_silent(model)
optimize!(model)

println("Termination Status: ", termination_status(model))
println("Optimal Cost: \$", objective_value(model))

Termination Status: OPTIMAL
Optimal Cost: $1200.0


# 5-4

In [63]:
using DataFrames, XLSX, NamedArrays
table = XLSX.readtable("alloy-blending.xlsx", "raw_data"; infer_eltypes=true)
df = DataFrame(table)

J = df.Raw 

# Get the cost of each raw
d = Dict(zip(J,df.Cost))

table = XLSX.readtable("alloy-blending.xlsx", "order_data"; infer_eltypes=true)
df = DataFrame(table)

I = df.Order 

# Now make dictionary for revenue lower and upper
r = Dict(zip(I,df.Revenue))
l = Dict(zip(I,df.Lower))
u = Dict(zip(I,df.Upper))

# Get the elements
table = XLSX.readtable("alloy-blending.xlsx", "element_data"; infer_eltypes=true)
df = DataFrame(table)
E = df.Element

# We'll make NamedArrays for the rest.  First initialize them all with zeros 
ei_zero = zeros(length(E), length(I))
α = NamedArray(ei_zero, (E,I), ("Element","Order"))
ei_zero2 = zeros(length(E), length(I))
β = NamedArray(ei_zero2, (E,I), ("Element","Order"))

table = XLSX.readtable("alloy-blending.xlsx", "element_order_data"; infer_eltypes=true)
df = DataFrame(table)
for row in eachrow(df)
    α[row.Element,row.Order] = row.Alpha 
    β[row.Element,row.Order] = row.Beta 
end 


ej_zero = zeros(length(E), length(J))
γ = NamedArray(ej_zero, (E,J), ("Element","Raw"))

table = XLSX.readtable("alloy-blending.xlsx", "element_raw_data"; infer_eltypes=true)
df = DataFrame(table)
for row in eachrow(df)
    γ[row.Element,row.Raw] = row.Gamma 
end 


In [65]:
model = Model(HiGHS.Optimizer)

# Decision Variables: x[i,j] = tons of raw material j used for order i
@variable(model, x[i in I, j in J] >= 0)

@objective(model, Max, 
    sum(r[i] * sum(x[i, j] for j in J) for i in I) -  # Revenue from orders
    sum(d[j] * sum(x[i, j] for i in I) for j in J)     # Cost of raw materials
)

# 1. Order Quantity Constraints
@constraint(model, [i in I], sum(x[i, j] for j in J) >= l[i])  
@constraint(model, [i in I], sum(x[i, j] for j in J) <= u[i])  

# 2. Element Composition Constraints
@constraint(model, [i in I, e in E],
    sum((γ[e, j] - α[e, i]) * x[i, j] for j in J) >= 0  # Min element requirement
)
@constraint(model, [i in I, e in E],
    sum((γ[e, j] - β[e, i]) * x[i, j] for j in J) <= 0  # Max element requirement
)
set_silent(model)
optimize!(model)

println("Termination Status: ", termination_status(model))
println("Optimal Profit: \$", objective_value(model))

Termination Status: OPTIMAL
Optimal Profit: $1239.3020188586154
